In [1]:
# 패키지 import

from konlpy.tag import Mecab, Kkma
from collections import Counter
import math
import numpy as np
import networkx as nx
import re

In [2]:

# 괄호 제거 (사이에 있는 내용도 제거) -------------------------------------------------------------------------------------
# 괄호 위치 반환 함수------------------------------------


def br_idx(s_br, e_br, txt):
    braket_start = txt.find(s_br)
    braket_end = txt.find(e_br)
    result = [braket_start, braket_end]
    return result


# r = br_idx("(", ")", text)
# print(r)
# print(text[0])
# --------------------------------------------------------
# 괄호 종류별 인덱스 추출

def rmTxt(text):
    rm_txt = text

    while True:
        if "(" in rm_txt:
            result = br_idx("(", ")", rm_txt)
            rm_txt = rm_txt[:result[0]] + rm_txt[result[1]+1:]
        elif "[" in rm_txt:
            result = br_idx("[", "]", rm_txt)
            rm_txt = rm_txt[:result[0]] + rm_txt[result[1]+1:]
        elif "<" in rm_txt:
            result = br_idx("<", ">", rm_txt)
            rm_txt = rm_txt[:result[0]] + rm_txt[result[1]+1:]
        if ("(" not in rm_txt) and ("[" not in rm_txt) and ("<" not in rm_txt):
            break
    return rm_txt

# print(rm_txt)
# ------------------------------------------------------------------------------------------------------

# 텍스트를 문장별로 쪼개기
# kkma = Kkma()
# sentences = kkma.sentences(text)

def sen(rm_txt):
    sentences = rm_txt.split(".")
    sp_words = ["'", '"', ",", "/", "\\", "=", "↓", "↑", "\n"]


    for idx, sentence in enumerate(sentences):
    #     print(idx)
        sp_idx = 0
        for sp_idx in range(len(sp_words)):
            sentences[idx-1] = sentences[idx-1].strip(" ")
            if sp_words[sp_idx] in sentence:
                sentences[idx] = sentences[idx].replace(sp_words[sp_idx], "")
            if "" in sentences:
                sentences.remove("")
                
#                 print("있음")
    return sentences
# print(sentences)

In [3]:
# Text Tokenizer
# 텍스트 토큰화(텍스트 전처리단계)

def token(text):
    mecab = Mecab(dicpath="C:\MeCab\mecab-ko-dic")
    # mecab은 dictionary 위치를 넣어줘야함

    # 명사 추출 테스트
    nouns = mecab.nouns(text)
    
    return nouns
# print(nouns)

In [4]:
# 단어 중복 배제 추출

# deduplicate_nouns = dedup_n

def dedup(nouns):
    dedup_n = []

    for noun in nouns:
    #     print(noun)
        if noun not in dedup_n:
            dedup_n.append(noun)
    return dedup_n
# 중복 배제된 단어 추출 완료
# print(dedup_n)

In [5]:
# tf 함수
# t : 단어, d : 문서 하나
# 문서 전체의 단어들 d와 단어들의 t의 빈도수
# f(t, d) 방식. f: frequency
# tf(t,d) = log (f(t,d) + 1);

def tf(t, d):
    freq = np.zeros(len(t)).reshape(len(t), 1)
#     print(len(d), len(t))
    for n in range(len(t)):
        count = 0
        for m in range(len(d)):
            if t[n] in d[m]:
                count += 1
                freq[n] = math.log(count)+1
                
    return freq

# tf test
# t = tf(dedup_n, nouns)
# print(t)

In [6]:
def idf(t, D):
    freq = np.zeros(len(t) * len(D)).reshape(len(t), len(D))
    for idx_D in range(len(D)):
        count = 0
        for idx_t in range(len(t)):
            if t[idx_t] in D[idx_D]:
                count += 1
            freq[idx_t][idx_D] = math.log(len(D)/(count+1))
    return freq

# idf test
# i = idf(dedup_n, sentences)
# print(i)

In [7]:
# tf-idf 구현

def tfidf(t, d, D):
    tfidf = tf(t, d) * idf(t, D)
    return tfidf

# tf-idf test
# ti = tfidf(dedup_n, nouns, sentences)
# print(ti)

In [8]:
# 가중치?

# print(weight)

def mk_graph(sentences):  
    graph = nx.Graph()
    graph.add_nodes_from(sentences)
    for x in range(len(sentences)):
        for y in range(len(sentences)):
            graph.add_edge(x, y, weight = weight[x, y])
    return graph


# print(reordered[0])

In [9]:
while True:
    # 요약할 텍스트 입력
    while True:
        try:
            title = input("요약할 텍스트의 타이틀을 입력해주세요\n>>> ")
            print()
            text = input("요약할 텍스트를 입력해주세요\n>>> ")
            if text and title:
                print()
                print("텍스트가 정상적으로 입력되었습니다.")
                print()
                break
            else:
                print()
                print("텍스트가 정상적으로 입력되지 않았습니다. 다시 한 번 입력해주세요")
                print()
        except:
            print("오류가 발생하였습니다")
            
    # ----------------------------------------------------------------------
    rm_txt = rmTxt(text)
    sentences = sen(rm_txt)
    nouns = token(text)
    dedup_n = dedup(nouns)
    ti = tfidf(dedup_n, nouns, sentences)
    weight = np.dot(ti.T, ti)
    graph = mk_graph(sentences)
    pagerank = nx.pagerank(graph, weight='weight')
    reordered = sorted(pagerank, key=pagerank.get, reverse=True)     
    # ----------------------------------------------------------------------
    
    try:
        n_sen = int(input(f"본문의 문장은 총 {len(sentences)}개 입니다.\n\n몇 문장으로 요약하시겠습니까?\n>>> "))
        print()
        if n_sen < len(sentences) and n_sen > 0:
            print("정상적으로 입력되었습니다")
            print()
        else:
            print(f"본문의 문장은 총 {len(sentences)}개 입니다.")
            print(f"{len(sentences)} ~ 0 사이의 숫자를 입력해주세요")
            print()
    except:
        print("오류")
        
    #------------------------------------------------------------------------------------------------
    
    print("요약이 완료되었습니다.")
    print("본문의 요약은 다음과 같습니다.")
    print()
    print()
    print(f"제목 : {title}")
    print("-"*80)
    print("내용 :")
    sent_temp = []
    for i in range(n_sen):
        sent_temp.append(reordered[i])

    # print(sent_temp)
    sent_asc = np.sort(np.array(sent_temp))
    # print(sent_asc)

    out_temp = []
    for i in sent_asc:
        out_temp.append(sentences[i])

    out = ".".join(out_temp)
    print(out)
    print("-"*80)
    
    # -----------------------------------------------------------------------------------------------
    
    finish = input("계속하시려면 아무 글자나 입력해주세요. 엔터키를 누르면 종료합니다\n>>>")
    if not finish:
        break

요약할 텍스트의 타이틀을 입력해주세요
>>> 미 '여름 시작' 메모리얼데이 연휴, 3천700만 명 이동 예상

요약할 텍스트를 입력해주세요
>>> 미국에서 코로나 19 규제가 완화되는 가운데 본격적인 여름 휴가철을 알리는 메모리얼 데이 연휴를 맞아 3천700만 명 이상이 장거리 여행에 나설 것으로 추산됐습니다.  USA투데이 등 미국 주요 언론은 현지시간 28일 전미자동차협회 발표를 인용해 이번 주말 시작되는 메모리얼 데이 연휴에 미국인 3천700만 명 이상이 집에서부터 최소 50마일, 80km 이상 떨어진 곳으로 여행할 예정이라고 전했습니다.  협회는 "코로나19 팬데믹이 한창이던 작년 메모리얼 데이 연휴와 비교해 여행객이 60% 이상 증가했다"며 팬데믹 이전인 2019년보다는 13% 적다고 설명했습니다.  여행객 대다수인 3천400만 명이 자동차를 이용한 도로 여행을 떠나고, 250만 명가량이 항공 여행을, 나머지 25만 명 정도가 버스와 기차 등 대중교통 수단을 이용할 계획입니다.  AP통신에 따르면 연휴 시작 전인 지난 27일, 미 전역의 공항에서 180만 명 이상이 항공편을 이용해 이동을 시작했습니다.

텍스트가 정상적으로 입력되었습니다.

본문의 문장은 총 5개 입니다.

몇 문장으로 요약하시겠습니까?
>>> 3

정상적으로 입력되었습니다

요약이 완료되었습니다.
본문의 요약은 다음과 같습니다.


제목 : 미 '여름 시작' 메모리얼데이 연휴, 3천700만 명 이동 예상
--------------------------------------------------------------------------------
내용 :
미국에서 코로나 19 규제가 완화되는 가운데 본격적인 여름 휴가철을 알리는 메모리얼 데이 연휴를 맞아 3천700만 명 이상이 장거리 여행에 나설 것으로 추산됐습니다.USA투데이 등 미국 주요 언론은 현지시간 28일 전미자동차협회 발표를 인용해 이번 주말 시작되는 메모리얼 데이 연휴에 미국인 3천700만 명 이상이 집에서부터 최소 5